# Table of Contents
 <p>

In [1]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>''')

In [2]:
################# IMPORTS 
###################################################
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display,clear_output
from pylayers.simul.link import *
import copy
import pylayers.util.geomutil as geu
import os
import tqdm
%gui wx
%matplotlib inline

def update_L(Lname):
    
#     from IPython.core.debugger import Tracer

#     Tracer()()
    DL.L=Layout(Lname)

    xmin,xmax,ymin,ymax=DL.L.ax
    zmax = DL.L.maxheight-0.1
    tx=copy.copy(DL.a)
    rx=copy.copy(DL.b)

    
    tx_x.min,tx_x.max=xmin,xmax
    tx_y.min,tx_y.max=ymin,ymax
    tx_z.max=zmax
    tx_x.value=tx[0]
    tx_y.value=tx[1]
    tx_z.value=tx[2]
    

    rx_x.min,rx_x.max=xmin,xmax
    rx_y.min,rx_y.max=ymin,ymax
    rx_z.max=zmax
    rx_x.value=rx[0]
    rx_y.value=rx[1]
    rx_z.value=rx[2]
    
################# FUNC def
###################################################
    
def update_Aa(ant):
    DL.Aa=Antenna(ant)
def update_Ab(ant):
    DL.Ab=Antenna(ant)
    
def update_a(x,y,z):
    DL.a=np.array([x,y,z])
def update_b(x,y,z):
    DL.b=np.array([x,y,z])

def update_Ta(alpha,beta,gamma):
    T = geu.MEulerAngle(alpha,beta=beta,gamma=gamma)
    DL.Ta=T
def update_Tb(alpha,beta,gamma):
    T = geu.MEulerAngle(alpha,beta=beta,gamma=gamma)
    DL.Tb=T
    
def update_fGHz(fm,fM,fs):
    DL.fGHz=np.linspace(fm,fM,fs)
    
def DLeval(*args):
#     clear_output()
    pbar = tqdm.tqdm_notebook(total=100)
    DL.eval(verbose=False,
            force=force.value,
            cutoff=cutoff.value,
            diffraction = diff.value,
            ra_vectorized = vect.value,
            ra_number_mirror_cf = mirror.value,
            applywav = appwav.value,
            progressbar=pbar)
    DL.R._show3()
    pbar.close()


################# interface 
###################################################
mlab.clf()
DL=DLink()
xmin,xmax,ymin,ymax=DL.L.ax
zmax = DL.L.maxheight-0.1
DL._show3()
# page 1  LAy
layout = widgets.Dropdown(description='File:',
                          options=os.listdir(basename +'/struc/ini/'),
                          value='defstr.ini',
                          )

layoutpage = widgets.Box(children=[layout])

_=widgets.interactive(update_L,Lname=layout)


# page 2 TX

##### TX position
tx_x = widgets.FloatSlider(description='x :',
                           
                           min=DL.L.ax[0],
                           max=DL.L.ax[1],
                           value=DL.a[0],
                           step=0.1,
                           padding=4)
tx_y = widgets.FloatSlider(description='y :',
                           min=DL.L.ax[2],
                           max=DL.L.ax[3],
                           value=DL.a[1],
                           step=0.1,
                           padding=4)
tx_z = widgets.FloatSlider(description='z :',
                           min=0.,
                           max=zmax,
                           value=DL.a[2],
                           step=0.1,
                           padding=4)



tx_pos = widgets.Box(children=[tx_x, tx_y,tx_z])

_=widgets.interactive(update_a,x=tx_x,y=tx_y,z=tx_z)


###### TX antenna
# ant default
av_ant = ['Omni','Gauss']

# ant file
antext= ['vsh3','sh3']
for fname in os.listdir(basename +'/ant'):
    if fname.split('.')[-1] in antext:
        av_ant.append(fname)

tx_ant = widgets.Dropdown(description='File:', value='Omni',options=av_ant)


#######TX Rotation

gammaTX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='x-roll')


betaTX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='y-roll')

alphaTX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='z-roll')


tx_rot = widgets.Box(children=[alphaTX,betaTX,gammaTX])

tx_antbox = widgets.Box(children=[tx_ant,tx_rot])

tx_postab = widgets.Tab(children=[tx_pos,tx_antbox])
tx_postab.set_title(0, 'Position')
tx_postab.set_title(1, 'Antenna')

tx_page = widgets.Box(children=[tx_postab])


_=widgets.interactive(update_Ta,alpha=alphaTX,beta=betaTX,gamma=gammaTX)
_=widgets.interactive(update_Aa,ant=tx_ant)



# page 2 RX



rx_x = widgets.FloatSlider(description='x :',
                           min=DL.L.ax[0],
                           max=DL.L.ax[1],
                           value=DL.b[0],
                           step=0.1,
                           padding=4)
rx_y = widgets.FloatSlider(description='y :',
                           min=DL.L.ax[2],
                           max=DL.L.ax[3],
                           value=DL.b[1],
                           step=0.1,
                           padding=4)
rx_z = widgets.FloatSlider(description='z :',
                           min=0.,
                           max=zmax,
                           value=DL.b[2],
                           step=0.1,
                           padding=4)


rx_pos = widgets.Box(children=[rx_x, rx_y,rx_z])
_=widgets.interactive(update_b,x=rx_x,y=rx_y,z=rx_z)



rx_ant = widgets.Dropdown(description='File:', value='Omni',options=av_ant ,padding=12)

#######RX Rotation


gammaRX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='x-roll')


betaRX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='y-roll')

alphaRX=widgets.FloatSlider(value=0,
                         min=-np.pi,
                          max=np.pi,
                          step=np.pi/50,
                        description='z-roll')

rx_rot = widgets.Box(children=[alphaRX,betaRX,gammaRX])

rx_antbox = widgets.Box(children=[rx_ant,rx_rot])

rx_postab = widgets.Tab(children=[rx_pos,rx_antbox])
rx_postab.set_title(0, 'Position')
rx_postab.set_title(1, 'Antenna')

rx_page = widgets.Box(children=[rx_postab])


_=widgets.interactive(update_Tb,alpha=alphaRX,beta=betaRX,gamma=gammaRX)
_=widgets.interactive(update_Ab,ant=rx_ant)

# page 3

fm = widgets.FloatText(description='fGHz min', value=DL.fGHz[0],padding=12)
fM = widgets.FloatText(description='fGHz max', value=DL.fGHz[-1],padding=12)
fs = widgets.FloatText(description='fGHz step', value=.1,padding=12)


param_page = widgets.Box(children=[fm,fM,fs])

_=widgets.interactive(update_fGHz,fm=fm,fM=fM,fs=fs)





###### page 4 adv param

force = widgets.Checkbox(description='force',value=False)
cutoff = widgets.IntText(description='Cutoff',value=DL.cutoff)
diff = widgets.Checkbox(description='diffractions',value=True)
vect = widgets.Checkbox(description='vect.rays',value=True)
mirror = widgets.IntText(description='mirror',value=1)
appwav = widgets.Checkbox(description='apply wavform',value=True)

adv_param_page = widgets.Box(children=[force,cutoff,appwav,mirror,diff,vect],padding=100)


tabs = widgets.Tab(children=[layoutpage,tx_page,rx_page,param_page,adv_param_page])
tabs.set_title(0, 'Layout')
tabs.set_title(1, 'Tx')
tabs.set_title(2, 'Rx')
tabs.set_title(3, 'Parameters')
tabs.set_title(4, 'Advanced Parameters')


# eval

eval_button = widgets.Button(description='Eval')
eval_button.on_click(DLeval)


interface = widgets.Box(children=[tabs,eval_button])


display(interface)



In [ ]:
DL=DLink()

In [ ]:
DL.R.I.D.A